In [1]:
!pip install surprise
!pip install scikit-surprise

In [2]:
%%time
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, coo_matrix
from tqdm.auto import tqdm
from collections import Counter
from surprise import Dataset, Reader
import time
from surprise import KNNWithZScore

CPU times: total: 297 ms
Wall time: 428 ms


c:\python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\track_artists.csv"
train_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\train"
test_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\test"
pred_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\surprize_artist_KNNWithZScore_target"
target_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\surprize_artist_KNNWithZScore_pred"

In [4]:
def calc_mrr(predict, answer):
    for i in range(len(predict)):
        if predict[i] == answer:
            return 1. / (i + 1)
    return 0

max_prediction_len = 100

def calc_score(target_path, predict_path):
    with open(target_path) as f:
        y_true = [int(x.strip()) for x in f.readlines()]

    with open(predict_path) as f:
        y_pred = [[int(x) for x in line.strip().split(' ')] for line in f.readlines()]

    mrr_score = 0
    for (pred, answer) in zip(y_pred, y_true):
        if len(pred) > max_prediction_len:
            raise ValueError('$maximum prediction length is {}, got {}$'.format(max_prediction_len, len(y_pred[i])))
        mrr_score += calc_mrr(pred, answer)

    print(f"MRR@100 = {(mrr_score / len(y_true)):.4f}")

In [5]:
%%time
df = pd.read_csv(df_path)
df.index = df['trackId']

CPU times: total: 46.9 ms
Wall time: 57.8 ms


In [6]:
%%time

list_coord = []
target_val = []
np.random.seed(42)  
val_id = np.random.choice(range(10000), size=1000, replace=False)
val_id.sort()
all_artist = set()
ls_artists = []

with open(train_path) as f:
    lines = f.readlines()
    idx = 0 
    for line in tqdm(lines):
        tracks = line.strip().split(' ')
        tracks_arr = np.array(tracks)
        tracks_arr = tracks_arr.astype(np.int32)
        artist_arr = np.array([df.at[track, 'artistId'] for track in tracks_arr])
        if idx in val_id:
            target_val.append(artist_arr[-1])
            artist_arr = artist_arr[:-1]
        ls_artists.extend(artist_arr)
        all_artist = all_artist.union(set(artist_arr))
        artist_cnt = Counter(artist_arr)
        list_coord_user = [(idx, i[0], i[1]) for i in artist_cnt.items()]
        list_coord.extend(list_coord_user)
        idx += 1
        if idx > 10000:
            break

  1%|▌                                                                       | 10000/1160084 [00:08<16:15, 1178.64it/s]

CPU times: total: 9.56 s
Wall time: 9.52 s


In [7]:
%%time

data = pd.DataFrame(list_coord, columns=['uid', 'iid', 'rating_row'])
data['rating_max'] = data.rating_row / data.groupby('uid')['rating_row'].transform('max')
data['rating_part'] = data.rating_row / data.groupby('uid')['rating_row'].transform('sum')

CPU times: total: 2.27 s
Wall time: 2.28 s


# Baseline

In [8]:
all_artist_cnt = [i[0] for i in Counter(ls_artists).most_common(100)]

result = [' '.join(map(str, all_artist_cnt)) + '\n' for _ in val_id]

with open(pred_path, 'w') as f:
    f.writelines(result)

with open(target_path, 'w') as f:
    for i in target_val:
        f.write(f'{i}\n')

calc_score(target_path, pred_path)

MRR@100 = 0.0144


# RATING_MAX

In [9]:
%%time

reader = Reader(rating_scale=(0, 1))
dataset = Dataset.load_from_df(data[['uid', 'iid', 'rating_max']], reader)
trainset = dataset.build_full_trainset()

CPU times: total: 562 ms
Wall time: 565 ms


## KNNWithZScore

In [10]:
def optimize(k=40, 
             min_k=1,
             user_based=True,
             name="pearson_baseline",
             verbose=True):

    start_time = time.time()
    
    sim_options = {
        "name": name,
        "user_based": user_based
    }
    
    algo = KNNWithZScore(k=k, min_k=min_k, sim_options=sim_options, verbose=True)
    
    algo.fit(trainset)
    
    print("--- fit: %s seconds ---" % (time.time() - start_time))
    
    pred = []
    for uid in tqdm(val_id):
        pred_user = []
        for iid in all_artist:
            score = algo.predict(uid=uid, iid=iid).est
            pred_user.append((iid, score))
        pred.append(pred_user)

    res = []
    for user in tqdm(pred):
        ls = sorted(user, key=lambda x: x[1])
        temp = [i[0] for i in ls[-100:]]
        temp.reverse()
        res.append(temp)

    result = [' '.join(map(str, i)) + '\n' for i in res]


    with open(pred_path, 'w') as f:
        f.writelines(result)

    with open(target_path, 'w') as f:
        for i in target_val:
            f.write(f'{i}\n')

    calc_score(target_path, pred_path)

In [11]:
for user_based in [False, True]:
    for name in ['cosine', 'msd', 'pearson', 'pearson_baseline']:
        for k in [10, 40, 80]:
            for min_k in [1, 10, 50]:
                print(f'user_based: {user_based}, name: {name}, k: {k}, min_k: {min_k}')
                optimize(k=k, 
                         min_k=min_k,
                         user_based=user_based,
                         name=name,
                         verbose=True)

user_based: False, name: cosine, k: 10, min_k: 1
Computing the cosine similarity matrix...
Done computing similarity matrix.
--- fit: 17.166242837905884 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:07<00:00, 129.38it/s]


MRR@100 = 0.0011
user_based: False, name: cosine, k: 10, min_k: 10
Computing the cosine similarity matrix...
Done computing similarity matrix.
--- fit: 17.394760847091675 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:07<00:00, 133.81it/s]


MRR@100 = 0.0002
user_based: False, name: cosine, k: 10, min_k: 50
Computing the cosine similarity matrix...
Done computing similarity matrix.
--- fit: 16.1471951007843 seconds ---


...

MRR@100 = 0.0088
user_based: True, name: pearson_baseline, k: 80, min_k: 10
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
--- fit: 14.218116521835327 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 247.70it/s]


MRR@100 = 0.0541
user_based: True, name: pearson_baseline, k: 80, min_k: 50
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
--- fit: 13.647122621536255 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 337.84it/s]


MRR@100 = 0.0487


# RATING_PART

In [12]:
%%time

reader = Reader(rating_scale=(0, 1))
dataset = Dataset.load_from_df(data[['uid', 'iid', 'rating_part']], reader)
trainset = dataset.build_full_trainset()

CPU times: total: 797 ms
Wall time: 790 ms


In [15]:
optimize(k=40, 
         min_k=10,
         user_based=True,
         name='pearson_baseline',
         verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
--- fit: 16.32907724380493 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 213.16it/s]


MRR@100 = 0.0579


## Best

In [16]:
%%time

reader = Reader(rating_scale=(0, 1))
dataset = Dataset.load_from_df(data[['uid', 'iid', 'rating_part']], reader)
trainset = dataset.build_full_trainset()

CPU times: total: 891 ms
Wall time: 885 ms


In [17]:
for k in [30, 40, 50]:
    for min_k in [5, 10, 20]:
        print(f'k: {k}, min_k: {min_k}')
        optimize(k=k, 
                 min_k=min_k,
                 user_based=True,
                 name='pearson_baseline',
                 verbose=True)

k: 30, min_k: 5
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
--- fit: 15.757535934448242 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 207.01it/s]


MRR@100 = 0.0438
k: 30, min_k: 10
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
--- fit: 14.087469100952148 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 234.67it/s]


MRR@100 = 0.0591
k: 30, min_k: 20
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
--- fit: 13.968966722488403 seconds ---


...

MRR@100 = 0.0421
k: 50, min_k: 10
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
--- fit: 12.178640842437744 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 321.02it/s]


MRR@100 = 0.0570
k: 50, min_k: 20
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
--- fit: 11.193395137786865 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 306.14it/s]


MRR@100 = 0.0549


In [18]:
optimize(k=30, 
         min_k=10,
         user_based=True,
         name='msd',
         verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 7.748192310333252 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 226.66it/s]


MRR@100 = 0.0302


# Final best
k: 30, min_k: 10<br>
Estimating biases using als...<br>
Computing the pearson_baseline similarity matrix...<br>
Done computing similarity matrix.<br>
--- fit: 14.087469100952148 seconds ---<br>
100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [09:31<00:00,  1.75it/s]<br>
100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 234.67it/s]<br>
MRR@100 = 0.0591